In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2
import os
import glob
from tqdm import tqdm
import time

import numpy as np
import random
%matplotlib inline
import matplotlib.image as mpimg

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
def augment_brightness_camera_images(image):
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = .5+np.random.uniform()
    print(random_bright)
    image1[:,:,2] = image1[:,:,2]*random_bright
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

def transform_image(img,ang_range,shear_range):#,trans_range):
    # Rotation

    ang_rot = np.random.uniform(ang_range)-ang_range/2
    rows,cols,ch = img.shape    
    Rot_M = cv2.getRotationMatrix2D((cols/2,rows/2),ang_rot,1)

    # Translation
    #tr_x = trans_range*np.random.uniform()-trans_range/2
    #tr_y = trans_range*np.random.uniform()-trans_range/2
    #Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])

    # Shear
    pts1 = np.float32([[5,5],[20,5],[5,20]])

    pt1 = 5+shear_range*np.random.uniform()-shear_range/2
    pt2 = 20+shear_range*np.random.uniform()-shear_range/2
    
    # Brightness 
    

    pts2 = np.float32([[pt1,5],[pt2,pt1],[5,pt2]])

    shear_M = cv2.getAffineTransform(pts1,pts2)
        
    img = cv2.warpAffine(img,Rot_M,(cols,rows))
    #img = cv2.warpAffine(img,Trans_M,(cols,rows))
    img = cv2.warpAffine(img,shear_M,(cols,rows))
    
    #img = augment_brightness_camera_images(img)
    
    return img

In [ ]:
ini_path = "/content/drive/My Drive/";
INPUT_DIR=os.path.join(ini_path);
SOURCE_DIR=os.path.join(INPUT_DIR,"final_report_images","source")
RESULT_DIR=os.path.join(INPUT_DIR,"final_report_images","transform")
print(SOURCE_DIR)
print(RESULT_DIR)

/content/drive/My Drive/final_report_images/source
/content/drive/My Drive/final_report_images/transform


In [ ]:
j=0
for file in tqdm(glob.glob(os.path.join(SOURCE_DIR,"*.png"))):
  img = cv2.imread(file)
  #j+=1
  for i in range(8):
    #img = increase_brightness(img,value=20)
    img = transform_image(img,5,5)
    filename = os.path.basename(file);
    name = os.path.splitext(filename)[0]+"_"+str(i+1)+'.png'
    print(name)
    cv2.imwrite(os.path.join(RESULT_DIR,name),img)
  #if j>1:
    #break

In [ ]:
#Brightness
def brightness(img, low, high):
    value = random.uniform(low, high)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype = np.float64)
    hsv[:,:,1] = hsv[:,:,1]*value
    hsv[:,:,1][hsv[:,:,1]>255]  = 255
    hsv[:,:,2] = hsv[:,:,2]*value 
    hsv[:,:,2][hsv[:,:,2]>255]  = 255
    hsv = np.array(hsv, dtype = np.uint8)
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return img

#Zoom
def fill(img, h, w):
    img = cv2.resize(img, (h, w), cv2.INTER_CUBIC)
    return img

def zoom(img, value):
    if value > 1 or value < 0:
        print('Value for zoom should be less than 1 and greater than 0')
        return img
    value = random.uniform(value, 1)
    h, w = img.shape[:2]
    h_taken = int(value*h)
    w_taken = int(value*w)
    h_start = random.randint(0, h-h_taken)
    w_start = random.randint(0, w-w_taken)
    img = img[h_start:h_start+h_taken, w_start:w_start+w_taken, :]
    img = fill(img, h, w)
    return img

#horizontal flip
def horizontal_flip(img, flag):
    if flag:
        return cv2.flip(img, 1)
    else:
        return img

#shear
def shear(img,shear_range):
  rows,cols,ch = img.shape
  pts1 = np.float32([[5,5],[20,5],[5,20]])
  pt1 = 5+shear_range*np.random.uniform()-shear_range/2
  pt2 = 20+shear_range*np.random.uniform()-shear_range/2
  pts2 = np.float32([[pt1,5],[pt2,pt1],[5,pt2]])
  shear_M = cv2.getAffineTransform(pts1,pts2)
  img = cv2.warpAffine(img,shear_M,(cols,rows))
  return img

#increase sharpness
#def inc_con(img):

#rotate
def rotate(img,ang_range):
  ang_rot = np.random.uniform(ang_range/2,ang_range)
  temp = np.random.randint(100)
  if (temp>50):
    ang_rot *= -1
  rows,cols,ch = img.shape    
  Rot_M = cv2.getRotationMatrix2D((cols/2,rows/2),ang_rot,1)
  img = cv2.warpAffine(img,Rot_M,(cols,rows))
  return img

In [ ]:
test_file = os.path.join(SOURCE_DIR,"KGlCXR_1_2.png")
name = "KGlCXR_1_2"
file_ext = ".png"
img = cv2.imread(test_file)
name2 = name+"1"+file_ext
cv2.imwrite(os.path.join(RESULT_DIR,name2),brightness(img,0.8,1.5))

name2 = name+"2"+file_ext
cv2.imwrite(os.path.join(RESULT_DIR,name2),zoom(img,0.75))
time.sleep(2)
name2 = name+"3"+file_ext
cv2.imwrite(os.path.join(RESULT_DIR,name2),horizontal_flip(img,1))
time.sleep(2)
name2 = name+"4"+file_ext
cv2.imwrite(os.path.join(RESULT_DIR,name2),shear(img,2.5))
time.sleep(2)

In [ ]:
types = ["*.png","*.jpg","*.jpeg"]
for i in range(0,3):
  for file in tqdm(glob.glob(os.path.join(SOURCE_DIR,types[i]))):
    #for i in range(1,7):
    #file = SOURCE_DIR+'/KGlCXR_'+str(i)+'_2.png'
    #print(file)
    #if(i<169):
      #i += 1
      #continue
    #filename = os.path.basename(file);
    #name = os.path.splitext(filename)[0]
    #if(name.find('b')!=-1 or name.find('h')!=-1 or name.find('r')!=-1):
      #os.remove(file)
    #continue
    img = cv2.imread(file)
    img_h = horizontal_flip(img,1)
    #img_b = brightness(img,1.2,2)
    #img_b_h = brightness(img_h,1.2,2)
    img_r = rotate(img,45)
    img_r_h = rotate(img_h,45)
    #img_r_b = rotate(img_b,60)
    #img_r_b_h = rotate(img_b_h,60)
    img_z = zoom(img,random.uniform(0.7,0.71))
    img_z_h = zoom(img_h,random.uniform(0.7,0.71))
    #img_z_b = zoom(img_b,random.uniform(0.7,0.9))
    #img_z_b_h = zoom(img_b_h,random.uniform(0.7,0.9))
    #img_s = shear(img,random.uniform(2,3))
    #img_s_h = shear(img_h,random.uniform(2,3))
    #img_s_b = shear(img_b,random.uniform(2,3))
    #img_s_b_h = shear(img_b_h,random.uniform(2,3))
    #img_s_z = shear(img_z,random.uniform(2,2.75))
    #img_s_z_h = shear(img_z_h,random.uniform(2,2.75))
    #img_s_z_b = shear(img_z_b,random.uniform(2,2.75))
    #img_s_z_b_h = shear(img_z_b_h,random.uniform(2,2.75))
    filename = os.path.basename(file);
    chgs = "o"
    name = os.path.splitext(filename)[0]+"_"+chgs+'.png'
    cv2.imwrite(os.path.join(RESULT_DIR,name),img)
    chgs = "h"
    name = os.path.splitext(filename)[0]+"_"+chgs+'.png'
    cv2.imwrite(os.path.join(RESULT_DIR,name),img_h)
    chgs = "b"
    name = os.path.splitext(filename)[0]+"_"+chgs+'.png'
    #cv2.imwrite(os.path.join(RESULT_DIR,name),img_b)
    chgs = "b_h"
    name = os.path.splitext(filename)[0]+"_"+chgs+'.png'
    #cv2.imwrite(os.path.join(RESULT_DIR,name),img_b_h)
    chgs = "r"
    name = os.path.splitext(filename)[0]+"_"+chgs+'.png'
    cv2.imwrite(os.path.join(RESULT_DIR,name),img_r)
    chgs = "r_h"
    name = os.path.splitext(filename)[0]+"_"+chgs+'.png'
    cv2.imwrite(os.path.join(RESULT_DIR,name),img_r_h)
    chgs = "r_b"
    name = os.path.splitext(filename)[0]+"_"+chgs+'.png'
    #cv2.imwrite(os.path.join(RESULT_DIR,name),img_r_b)
    chgs = "r_b_h"
    name = os.path.splitext(filename)[0]+"_"+chgs+'.png'
    #cv2.imwrite(os.path.join(RESULT_DIR,name),img_r_b_h)
    chgs = "z"
    name = os.path.splitext(filename)[0]+"_"+chgs+'.png'
    cv2.imwrite(os.path.join(RESULT_DIR,name),img_z)
    chgs = "z_h"
    name = os.path.splitext(filename)[0]+"_"+chgs+'.png'
    cv2.imwrite(os.path.join(RESULT_DIR,name),img_z_h)
    chgs = "s"
    name = os.path.splitext(filename)[0]+"_"+chgs+'.png'
    #cv2.imwrite(os.path.join(RESULT_DIR,name),img_s)
    chgs = "s_h"
    name = os.path.splitext(filename)[0]+"_"+chgs+'.png'
    #cv2.imwrite(os.path.join(RESULT_DIR,name),img_s_h)
    chgs = "s_b"
    name = os.path.splitext(filename)[0]+"_"+chgs+'.png'
    #cv2.imwrite(os.path.join(RESULT_DIR,name),img_s_b)
    chgs = "s_b_h"
    name = os.path.splitext(filename)[0]+"_"+chgs+'.png'
    #cv2.imwrite(os.path.join(RESULT_DIR,name),img_s_b_h)
    chgs = "s_z"
    name = os.path.splitext(filename)[0]+"_"+chgs+'.png'
    #cv2.imwrite(os.path.join(RESULT_DIR,name),img_s_z)
    chgs = "s_z_h"
    name = os.path.splitext(filename)[0]+"_"+chgs+'.png'
    #cv2.imwrite(os.path.join(RESULT_DIR,name),img_s_z_h)
    chgs = "s_z_b"
    name = os.path.splitext(filename)[0]+"_"+chgs+'.png'
    #cv2.imwrite(os.path.join(RESULT_DIR,name),img_s_z_b)
    chgs = "s_z_b_h"
    name = os.path.splitext(filename)[0]+"_"+chgs+'.png'
    #cv2.imwrite(os.path.join(RESULT_DIR,name),img_s_z_b_h)

100%|██████████| 1/1 [00:00<00:00,  1.95it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
